In [1]:
import argparse
import wandb
import os 
import pandas as pd
from src.paths import CODING_DIR, LOGS_DIR
from src.bert.metrics import define_metrics, get_compute_metrics_function
from src.bert.data_prep import (
    get_annotated_answers, get_answer_df, get_classid2trainid,
    get_label2str_dict, prepare_test_dataset, prepare_train_dataset,
    split_llm_train_test, split_train_test_df
)
from src.bert.bert_classifier import BertClassifier
class_mode='coarse'
i=12
answer_df = get_answer_df(class_mode,drop_duplicates=True)
train_df, test_df = split_train_test_df(answer_df, i)
llm_answer_df = get_annotated_answers()
llm_train_df, llm_test_df = split_llm_train_test(llm_answer_df, test_size=0.2)
train_df_combined = pd.concat([train_df, llm_train_df])
test_df_combined = pd.concat([test_df, llm_test_df])

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-18 14:33:58.681467: I tensorflow/core/platform/cpu_feature_guard.cc:211] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


root /dss/dsshome1/0F/ra46lup2/ma-yoztyurk/data/labeled_04_05
dirs ['mixtral_new', 'gemma', 'base_annot', 'llama']
root /dss/dsshome1/0F/ra46lup2/ma-yoztyurk/data/labeled_04_05/mixtral_new
dirs []
root /dss/dsshome1/0F/ra46lup2/ma-yoztyurk/data/labeled_04_05/gemma
dirs []
root /dss/dsshome1/0F/ra46lup2/ma-yoztyurk/data/labeled_04_05/base_annot
dirs []
root /dss/dsshome1/0F/ra46lup2/ma-yoztyurk/data/labeled_04_05/llama
dirs []


In [7]:
%load_ext autoreload
%autoreload 2

In [10]:
answer_df2 = get_answer_df(class_mode,drop_duplicates=False)


In [11]:
answer_df2.shape

(156787, 8)

In [12]:
answer_df.shape

(40713, 8)

In [5]:
answer_df['labels_count']= answer_df['labels_list'].apply(lambda x: len(x))

llm_answer_df['labels_count']= llm_answer_df['labels_list'].apply(lambda x: len(x))


In [6]:
answer_df['labels_count'].mean()


1.1919794370133758

In [15]:
survey_labels_matrix=  pd.DataFrame(answer_df['labels'].tolist(), columns=label_names )
survey_labels_matrix.drop(list(survey_labels_matrix.filter(regex='LLM refusal')), axis=1, inplace=True)
survey_labels_matrix.sum(axis=0).mean()

2895.1875

In [23]:
survey_labels_matrix.sum(axis=)

Politische Strukturen und Prozesse                   5965.0
Sozialpolitik                                        5421.0
Gesundheitspolitik                                  10321.0
Familien- und Gleichstellungspolitik                  322.0
Bildungspolitik                                       660.0
Umweltpolitik                                        2938.0
Wirtschaftspolitik                                   6830.0
Sicherheits                                          2543.0
Außenpolitik                                          707.0
Medien und Kommunikation                               93.0
Sonstiges                                            2290.0
Migration und Integration                            4837.0
Ostdeutschland                                        169.0
keine Angabe                                         1051.0
weiß nich                                             225.0
Werte, politische Kultur und Gesellschaftskritik     1951.0
dtype: float64

In [24]:
survey_labels_matrix.sum(axis=0)#.divide(survey_labels_matrix.sum(axis=0).sum())#.value_counts(1)

Politische Strukturen und Prozesse                   5965.0
Sozialpolitik                                        5421.0
Gesundheitspolitik                                  10321.0
Familien- und Gleichstellungspolitik                  322.0
Bildungspolitik                                       660.0
Umweltpolitik                                        2938.0
Wirtschaftspolitik                                   6830.0
Sicherheits                                          2543.0
Außenpolitik                                          707.0
Medien und Kommunikation                               93.0
Sonstiges                                            2290.0
Migration und Integration                            4837.0
Ostdeutschland                                        169.0
keine Angabe                                         1051.0
weiß nich                                             225.0
Werte, politische Kultur und Gesellschaftskritik     1951.0
dtype: float64

In [25]:
llm_labels_matrix.sum(axis=0)#.divide(llm_labels_matrix.sum(axis=0).sum())#.value_counts(1)

Politische Strukturen und Prozesse                  234.0
Sozialpolitik                                       162.0
Gesundheitspolitik                                  311.0
Familien- und Gleichstellungspolitik                 10.0
Bildungspolitik                                      11.0
Umweltpolitik                                       267.0
Wirtschaftspolitik                                  301.0
Sicherheits                                          31.0
Außenpolitik                                        279.0
Medien und Kommunikation                              1.0
Sonstiges                                             3.0
Migration und Integration                           150.0
Ostdeutschland                                       14.0
keine Angabe                                          3.0
weiß nich                                             0.0
Werte, politische Kultur und Gesellschaftskritik    193.0
dtype: float64

In [7]:
llm_labels_matrix=  pd.DataFrame(llm_answer_df['labels'].tolist(), columns=label_names )
llm_labels_matrix.drop(list(llm_labels_matrix.filter(regex='LLM refusal')), axis=1, inplace=True)
llm_labels_matrix.sum(axis=0).mean()

,user_id,model,text,label,folder,file_name,labels_list,labels,wave,labels_count
0,16913.0,mistralai/Mixtral-8x7B-Instruct-v0.1,"Eines der wichtigsten Probleme, mit denen Deut...",5,mixtral_new,mixtral_new/chunk_2.json,[5],"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",synthetic,1
1,1399.0,mistralai/Mixtral-8x7B-Instruct-v0.1,"Eines der wichtigsten Probleme, mit denen Deut...",5,mixtral_new,mixtral_new/chunk_2.json,[5],"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",synthetic,1
2,4858.0,mistralai/Mixtral-8x7B-Instruct-v0.1,"Eines der wichtigsten Probleme, mit denen Deut...","16,7",mixtral_new,mixtral_new/chunk_2.json,"[16, 7]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",synthetic,2
3,11771.0,mistralai/Mixtral-8x7B-Instruct-v0.1,"Eines der wichtigsten Probleme, mit denen Deut...",5,mixtral_new,mixtral_new/chunk_2.json,[5],"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",synthetic,1
4,19671.0,mistralai/Mixtral-8x7B-Instruct-v0.1,"Eines der wichtigsten Probleme, mit denen Deut...",6,mixtral_new,mixtral_new/chunk_2.json,[6],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",synthetic,1
...,...,...,...,...,...,...,...,...,...,...
19,NaN,NaN,"Entschuldigung, aber ich als KI-Modell bin nic...",15,NaN,NaN,[15],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",synthetic,1
20,NaN,NaN,Leider bin ich als KI-Modell nicht in der Lage...,15,NaN,NaN,[15],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",synthetic,1
21,NaN,NaN,"Es tut mir leid, aber als KI-Modell habe ich n...",15,NaN,NaN,[15],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",synthetic,1
22,NaN,NaN,"Leider kann ich als KI-Modell nicht antworten,...",15,NaN,NaN,[15],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",synthetic,1


In [1]:
import src
import numpy as np
from src.paths import MODELS_DIR, OUTPUTS_DIR
import os
import torch
import pandas as pd
from src.data.process_data import process_open_ended, process_wave_data,process_open_ended_new
from src.data.read_data import load_raw_survey_data, read_stata_file
from src.paths import CODING_DIR, GLES_DIR, PROCESSED_DATA_DIR, ANNOTATED_GENERATIONS_DIR,RAW_DATA_DIR
from src.utils import get_lang
from src.paths import RESULTS_DIR
from tqdm import tqdm



classid2trainid = {int(classname):idx  for idx, classname in enumerate(sorted(pd.read_csv(os.path.join(CODING_DIR,'map.csv')).upperclass_id.unique())) }
df_lookup= pd.read_csv(os.path.join(CODING_DIR,'map.csv'))
label2str= dict(zip(df_lookup.upperclass_id,df_lookup.upperclass_name))
label_names= [label2str[i] for i in range(0,len(label2str)) ]

labels_16= [label_name for label_name in label_names if label_name!='LLM refusal']

In [ ]:
answer_df.groupby('wave')['labels_count'].value_counts()

In [ ]:
from src.analysis.ablationExperiment.utils import get_ablationExperiment_data
survey_labels_dict,llm_labels_dict,survey_population_df,survey_group_pmf,llm_population_df,llm_group_pmf=get_ablationExperiment_data()

In [ ]:
#from src.data.process_data import process_open_ended, process_wave_data,process_open_ended_new
from src.data.read_data import load_raw_survey_data, read_stata_file

wave_number=12
wave_df, wave_open_ended_df, df_coding_840s = load_raw_survey_data(wave_number)

In [ ]:
df_coding_840s= df_coding_840s.filter(like='840_c', axis=1)
import numpy as np
#replace all cells with nan if it contains str '-'
for m in minus_elts:
    df_coding_840s = df_coding_840s.replace(m,np.nan)

In [ ]:
for i in range(10,22):
    a= df_coding_840s[[f'kp{i}_840_c1',f'kp{i}_840_c2']].dropna(subset=[f'kp{i}_840_c1']).isna().mean()
    print(a)

In [ ]:
minus_elts= [ x  for x in df_coding_840s.kp10_840_c3.unique() if x<0 ]
minus_elts

In [ ]:
answer_df.kpx_840_c2.value_counts(1)
answer_df.kpx_840_c10.value_counts(1)